<a href="https://colab.research.google.com/github/grace3999/PPs/blob/master/Kaggle_GOSSIS_dataprep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp
import missingno as msno
import pickle

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})

#machine learning
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import StandardScaler, Normalizer, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
#data paths
train_data_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/produced/train_data_mine_noimpute.csv'
unlabeled_data_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/produced/test_data_mine_noimpute.csv'
feature_imp_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/produced/feature_imp.csv'
#first 39 imp data files
train39_data_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/produced/train_data_39featH2O_impute.csv'
unlabeled39_data_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/produced/test_data_39featH2O_impute.csv'

# Train data first


In [9]:
train_supplied = pd.read_csv(train_data_path)
train_supplied = pd.DataFrame(data = train_supplied)

print('Original data shape:\n', train_supplied.shape, '\n')
print('Group value counts:\n', train_supplied['hospital_death'].value_counts(), '\n')

train39_supplied = pd.read_csv(train39_data_path)
train39_supplied = pd.DataFrame(data = train39_supplied)

print('Original data shape:\n', train39_supplied.shape, '\n')
print('Group value counts:\n', train39_supplied['hospital_death'].value_counts(), '\n')

train39_supplied.head()

Original data shape:
 (91713, 153) 

Group value counts:
 0    83798
1     7915
Name: hospital_death, dtype: int64 

Original data shape:
 (91713, 41) 

Group value counts:
 0    83798
1     7915
Name: hospital_death, dtype: int64 



,Unnamed: 0,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,apache_3j_diagnosis,d1_spo2_min,d1_hco3_min,apache_2_diagnosis,bmi,d1_heartrate_min,d1_sysbp_min,d1_mbp_min,ventilated_apache,d1_platelets_min,d1_resprate_min,param_10_diff_count,d1_temp_min,param_25_diff_count,d1_platelets_max,d1_hco3_max,wbc_apache,age,weight,d1_spo2_diff,d1_temp_max,heart_rate_diff,pre_icu_los_days,creatinine_apache,h1_temp_max,d1_wbc_min,d1_glucose_min,d1_resprate_max,d1_sysbp_max,d1_bun_min,d1_wbc_max,d1_diasbp_min,h1_resprate_min,map_apache,temp_diff,temp_apache,glucose_apache,hospital_death
0,0,0.100000,0.050000,0.104856,74.0,15.000000,0.157922,22.730000,72.0,73.0,46.0,0.0,233.000000,10.0,23.0,37.2,13.0,233.000000,19.000000,14.100000,68.0,73.900000,26.0,39.9,47.0,0.541667,2.510000,39.500000,14.100000,109.000000,34.0,131.0,30.000000,14.100000,37.0,18.000000,40.0,2.7,39.3,168.000000,0
1,1,0.470000,0.290000,0.197111,70.0,26.000000,0.197111,27.420000,72.0,67.0,38.0,1.0,487.000000,12.0,8.0,35.1,0.0,557.000000,27.000000,12.700000,77.0,70.200000,30.0,36.3,46.0,0.927778,0.560000,36.300000,12.700000,128.000000,32.0,159.0,9.000000,23.300000,31.0,28.000000,46.0,1.2,35.1,145.000000,0
2,2,-0.078623,-0.051812,0.007727,91.0,23.476139,0.008493,31.950000,68.0,105.0,68.0,0.0,231.032011,8.0,38.0,36.7,15.0,232.173462,24.492458,9.879730,25.0,95.300000,7.0,37.0,28.0,0.000694,1.242159,36.700000,9.944959,112.449489,21.0,148.0,11.797425,9.940814,48.0,16.000000,68.0,0.3,36.7,157.604632,0
3,3,0.040000,0.030000,0.019481,95.0,20.131523,0.028886,22.640000,92.0,84.0,84.0,1.0,43.000000,7.0,8.0,34.8,2.0,198.000000,24.238908,8.000000,81.0,61.700000,5.0,38.0,24.0,0.000694,1.370443,35.600000,8.000000,88.000000,23.0,158.0,20.041341,9.000000,42.0,11.000000,60.0,3.2,34.8,185.000000,0
4,4,-0.040290,-0.017745,0.091518,96.0,23.756070,0.087740,31.474937,60.0,120.0,90.0,0.0,239.412553,16.0,38.0,36.7,15.0,238.901566,24.627807,9.822814,19.0,92.294235,4.0,37.2,29.0,0.073611,1.115602,36.990701,9.912383,110.312610,18.0,147.0,8.468848,9.798405,57.0,17.299467,103.0,0.5,36.7,146.845870,0


### get feature sets from feature importance 


In [10]:
feature_imp = pd.read_csv(feature_imp_path)
feature_imp = pd.DataFrame(data = feature_imp)
print(feature_imp.shape)
feature_imp.head()


(151, 24)


,feature,lr_scaled_balanced_importance,lr_scaled_NObalanced_importance,lgb_NOscaled_balanced_importance,lgb_NOscaled_NObalanced_importance,lgb_NOscaled_balanced_withna_importance,lgb_NOscaled_NObalanced_withna_importance,relative_importance,scaled_importance,percentage,variable,relative_importance.1,scaled_importance.1,percentage.1,Unnamed: 14,Unnamed: 15,Unnamed: 16,lr_bal,lr_nobal,lgb_bal,lgb_nobal,lgb_bal_withna,lgb_nobal_withna,autoH2O_nobal
0,age,0.235557,0.153373,37,26,77,103,503.420654,0.038834,0.008601,ventilated_apache,1686.562988,0.049841,0.009048,NaN,NaN,NaN,apache_3j_diagnosis,apache_4a_hospital_death_prob,apache_4a_hospital_death_prob,apache_4a_hospital_death_prob,apache_4a_hospital_death_prob,apache_3j_diagnosis,apache_4a_hospital_death_prob
1,age_score,0.063798,0.052735,0,0,0,0,132.341400,0.010209,0.002261,na_count,494.488251,0.014613,0.002653,NaN,NaN,NaN,apache_4a_hospital_death_prob,apache_3j_diagnosis,apache_3j_diagnosis,d1_heartrate_min,apache_3j_diagnosis,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
2,aids,-0.013472,0.003579,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d1_diasbp_noninvasive_min,d1_diasbp_noninvasive_min,apache_4a_icu_death_prob,apache_3j_diagnosis,age,age,apache_3j_diagnosis
3,apache_2_bodysystem,0.128392,0.095073,3,3,5,13,58.315002,0.004498,0.000996,d1_hco3_diff,233.147690,0.006890,0.001251,NaN,NaN,NaN,ventilated_apache,creatinine_apache,d1_heartrate_min,apache_4a_icu_death_prob,d1_heartrate_min,d1_heartrate_min,d1_spo2_min
4,apache_2_diagnosis,-0.504262,-0.436540,10,13,23,54,789.365478,0.060892,0.013486,d1_platelets_min,2843.425537,0.084028,0.015254,NaN,NaN,NaN,d1_mbp_min,hematocrit_apache,age,d1_platelets_min,apache_4a_icu_death_prob,pre_icu_los_days,d1_hco3_min


In [0]:
# features from H2O
H2O_feat_imp = feature_imp.sort_values('relative_importance', ascending=False)['feature']
feat_H2O_first39 = H2O_feat_imp[0:39].values
feat_H2O_second39 = H2O_feat_imp[39:78].values

### impute missing data

In [13]:
imp_features = feat_H2O_second39

#create feature set 
train_supplied_imp1forimp2 = pd.concat([train39_supplied.drop(['Unnamed: 0', 'hospital_death'], axis=1), train_supplied[feat_H2O_second39]], axis=1)
print(train_supplied_imp1forimp2.shape)
train_supplied_imp1forimp2.head()


(91713, 78)


,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,apache_3j_diagnosis,d1_spo2_min,d1_hco3_min,apache_2_diagnosis,bmi,d1_heartrate_min,d1_sysbp_min,d1_mbp_min,ventilated_apache,d1_platelets_min,d1_resprate_min,param_10_diff_count,d1_temp_min,param_25_diff_count,d1_platelets_max,d1_hco3_max,wbc_apache,age,weight,d1_spo2_diff,d1_temp_max,heart_rate_diff,pre_icu_los_days,creatinine_apache,h1_temp_max,d1_wbc_min,d1_glucose_min,d1_resprate_max,d1_sysbp_max,d1_bun_min,d1_wbc_max,d1_diasbp_min,h1_resprate_min,map_apache,temp_diff,temp_apache,glucose_apache,d1_bun_max,h1_temp_min,resprate_apache,d1_sodium_max,height,d1_creatinine_min,d1_calcium_min,d1_potassium_max,bun_apache,d1_glucose_max,d1_sysbp_noninvasive_min,h1_heartrate_min,d1_diasbp_diff,d1_potassium_min,d1_heartrate_max,d1_creatinine_max,hematocrit_apache,d1_hemaglobin_max,d1_sysbp_noninvasive_max,resprate_diff,d1_hematocrit_max,d1_calcium_max,na_count,d1_sodium_min,h1_heartrate_max,h1_sysbp_min,sodium_apache,map_diff,h1_spo2_min,h1_resprate_max,d1_mbp_max,glucose_diff,d1_diasbp_max,heart_rate_apache,d1_mbp_noninvasive_min,d1_diasbp_noninvasive_min,d1_hematocrit_min,d1_sysbp_diff,d1_hemaglobin_min
0,0.100000,0.050000,0.104856,74.0,15.000000,0.157922,22.730000,72.0,73.0,46.0,0.0,233.000000,10.0,23.0,37.2,13.0,233.000000,19.000000,14.100000,68.0,73.900000,26.0,39.9,47.0,0.541667,2.510000,39.500000,14.100000,109.000000,34.0,131.0,30.000000,14.100000,37.0,18.000000,40.0,2.7,39.3,168.000000,31.0,37.5,36.0,136.0,180.3,2.23,7.4,4.0,31.0,168.0,73.0,108.0,31.0,3.4,119.0,2.51,27.4,8.9,131.0,24.0,27.4,8.5,60,134.0,119.0,115.0,134.0,43.0,74.0,26.0,89.0,59.0,68.0,118.0,46.0,37.0,27.4,58.0,8.9
1,0.470000,0.290000,0.197111,70.0,26.000000,0.197111,27.420000,72.0,67.0,38.0,1.0,487.000000,12.0,8.0,35.1,0.0,557.000000,27.000000,12.700000,77.0,70.200000,30.0,36.3,46.0,0.927778,0.560000,36.300000,12.700000,128.000000,32.0,159.0,9.000000,23.300000,31.0,28.000000,46.0,1.2,35.1,145.000000,11.0,36.3,33.0,145.0,160.0,0.56,8.0,4.2,9.0,145.0,67.0,100.0,64.0,3.8,118.0,0.71,36.9,11.3,159.0,20.0,36.9,8.6,19,145.0,114.0,71.0,145.0,82.0,70.0,31.0,120.0,17.0,95.0,120.0,38.0,31.0,36.1,92.0,11.1
2,-0.078623,-0.051812,0.007727,91.0,23.476139,0.008493,31.950000,68.0,105.0,68.0,0.0,231.032011,8.0,38.0,36.7,15.0,232.173462,24.492458,9.879730,25.0,95.300000,7.0,37.0,28.0,0.000694,1.242159,36.700000,9.944959,112.449489,21.0,148.0,11.797425,9.940814,48.0,16.000000,68.0,0.3,36.7,157.604632,NaN,36.7,37.0,NaN,172.7,NaN,NaN,NaN,NaN,NaN,105.0,78.0,40.0,NaN,96.0,NaN,NaN,NaN,148.0,13.0,NaN,NaN,104,NaN,96.0,124.0,NaN,34.0,91.0,20.0,102.0,NaN,88.0,102.0,68.0,48.0,NaN,43.0,NaN
3,0.040000,0.030000,0.019481,95.0,20.131523,0.028886,22.640000,92.0,84.0,84.0,1.0,43.000000,7.0,8.0,34.8,2.0,198.000000,24.238908,8.000000,81.0,61.700000,5.0,38.0,24.0,0.000694,1.370443,35.600000,8.000000,88.000000,23.0,158.0,20.041341,9.000000,42.0,11.000000,60.0,3.2,34.8,185.000000,NaN,34.8,4.0,NaN,165.1,NaN,NaN,5.0,NaN,185.0,84.0,96.0,6.0,3.5,116.0,NaN,25.9,11.6,158.0,16.0,34.0,NaN,48,NaN,100.0,106.0,NaN,0.0,99.0,12.0,84.0,97.0,48.0,114.0,84.0,42.0,25.9,74.0,8.9
4,-0.040290,-0.017745,0.091518,96.0,23.756070,0.087740,31.474937,60.0,120.0,90.0,0.0,239.412553,16.0,38.0,36.7,15.0,238.901566,24.627807,9.822814,19.0,92.294235,4.0,37.2,29.0,0.073611,1.115602,36.990701,9.912383,110.312610,18.0,147.0,8.468848,9.798405,57.0,17.299467,103.0,0.5,36.7,146.845870,NaN,NaN,16.0,NaN,188.0,NaN,NaN,NaN,NaN,NaN,120.0,76.0,42.0,NaN,89.0,NaN,NaN,NaN,147.0,2.0,NaN,NaN,115,NaN,89.0,120.0,NaN,14.0,100.0,NaN,104.0,NaN,99.0,60.0,90.0,57.0,NaN,27.0,NaN


In [14]:
data = train_supplied_imp1forimp2

imp = IterativeImputer(max_iter=99, random_state=39, verbose=2).fit(data)
train_supplied_imp = imp.transform(data)

#create final data set
final_data_train = pd.DataFrame(data=train_supplied_imp, columns=data.columns.values)
#put back correct hospital death
final_data_train['hospital_death'] = train_supplied['hospital_death']

final_data_train.head()

[IterativeImputer] Completing matrix with shape (91713, 78)
[IterativeImputer] Ending imputation round 1/99, elapsed time 72.83
[IterativeImputer] Change: 467.1629434176414, scaled tolerance: 0.611 
[IterativeImputer] Ending imputation round 2/99, elapsed time 153.09
[IterativeImputer] Change: 196.616467208908, scaled tolerance: 0.611 
[IterativeImputer] Ending imputation round 3/99, elapsed time 231.24
[IterativeImputer] Change: 38.451164331593894, scaled tolerance: 0.611 
[IterativeImputer] Ending imputation round 4/99, elapsed time 318.08
[IterativeImputer] Change: 54.3305554277117, scaled tolerance: 0.611 
[IterativeImputer] Ending imputation round 5/99, elapsed time 396.34
[IterativeImputer] Change: 73.93093106708585, scaled tolerance: 0.611 
[IterativeImputer] Ending imputation round 6/99, elapsed time 481.52
[IterativeImputer] Change: 160.40652693818777, scaled tolerance: 0.611 
[IterativeImputer] Ending imputation round 7/99, elapsed time 560.59
[IterativeImputer] Change: 59.15

,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,apache_3j_diagnosis,d1_spo2_min,d1_hco3_min,apache_2_diagnosis,bmi,d1_heartrate_min,d1_sysbp_min,d1_mbp_min,ventilated_apache,d1_platelets_min,d1_resprate_min,param_10_diff_count,d1_temp_min,param_25_diff_count,d1_platelets_max,d1_hco3_max,wbc_apache,age,weight,d1_spo2_diff,d1_temp_max,heart_rate_diff,pre_icu_los_days,creatinine_apache,h1_temp_max,d1_wbc_min,d1_glucose_min,d1_resprate_max,d1_sysbp_max,d1_bun_min,d1_wbc_max,d1_diasbp_min,h1_resprate_min,map_apache,temp_diff,temp_apache,glucose_apache,d1_bun_max,h1_temp_min,resprate_apache,d1_sodium_max,height,d1_creatinine_min,d1_calcium_min,d1_potassium_max,bun_apache,d1_glucose_max,d1_sysbp_noninvasive_min,h1_heartrate_min,d1_diasbp_diff,d1_potassium_min,d1_heartrate_max,d1_creatinine_max,hematocrit_apache,d1_hemaglobin_max,d1_sysbp_noninvasive_max,resprate_diff,d1_hematocrit_max,d1_calcium_max,na_count,d1_sodium_min,h1_heartrate_max,h1_sysbp_min,sodium_apache,map_diff,h1_spo2_min,h1_resprate_max,d1_mbp_max,glucose_diff,d1_diasbp_max,heart_rate_apache,d1_mbp_noninvasive_min,d1_diasbp_noninvasive_min,d1_hematocrit_min,d1_sysbp_diff,d1_hemaglobin_min,hospital_death
0,0.100000,0.050000,0.104856,74.0,15.000000,0.157922,22.730000,72.0,73.0,46.0,0.0,233.000000,10.0,23.0,37.2,13.0,233.000000,19.000000,14.100000,68.0,73.900000,26.0,39.9,47.0,0.541667,2.510000,39.500000,14.100000,109.000000,34.0,131.0,30.000000,14.100000,37.0,18.000000,40.0,2.7,39.3,168.000000,31.000000,37.50000,36.0,136.000000,180.3,2.230000,7.400000,4.000000,31.000000,168.000000,73.0,108.0,31.0,3.400000,119.0,2.510000,27.400000,8.900000,131.0,24.0,27.400000,8.500000,60.0,134.000000,119.0,115.0,134.000000,43.0,74.0,26.000000,89.0,59.000000,68.0,118.0,46.0,37.0,27.400000,58.0,8.900000,0
1,0.470000,0.290000,0.197111,70.0,26.000000,0.197111,27.420000,72.0,67.0,38.0,1.0,487.000000,12.0,8.0,35.1,0.0,557.000000,27.000000,12.700000,77.0,70.200000,30.0,36.3,46.0,0.927778,0.560000,36.300000,12.700000,128.000000,32.0,159.0,9.000000,23.300000,31.0,28.000000,46.0,1.2,35.1,145.000000,11.000000,36.30000,33.0,145.000000,160.0,0.560000,8.000000,4.200000,9.000000,145.000000,67.0,100.0,64.0,3.800000,118.0,0.710000,36.900000,11.300000,159.0,20.0,36.900000,8.600000,19.0,145.000000,114.0,71.0,145.000000,82.0,70.0,31.000000,120.0,17.000000,95.0,120.0,38.0,31.0,36.100000,92.0,11.100000,0
2,-0.078623,-0.051812,0.007727,91.0,23.476139,0.008493,31.950000,68.0,105.0,68.0,0.0,231.032011,8.0,38.0,36.7,15.0,232.173462,24.492458,9.879730,25.0,95.300000,7.0,37.0,28.0,0.000694,1.242159,36.700000,9.944959,112.449489,21.0,148.0,11.797425,9.940814,48.0,16.000000,68.0,0.3,36.7,157.604632,11.808896,36.70000,37.0,137.766503,172.7,1.192526,8.427179,3.958307,12.305896,138.941792,105.0,78.0,40.0,3.954041,96.0,1.226565,36.071884,12.133199,148.0,13.0,36.241038,8.403353,104.0,137.740283,96.0,124.0,137.269568,34.0,91.0,20.000000,102.0,26.492279,88.0,102.0,68.0,48.0,36.131365,43.0,12.088817,0
3,0.040000,0.030000,0.019481,95.0,20.131523,0.028886,22.640000,92.0,84.0,84.0,1.0,43.000000,7.0,8.0,34.8,2.0,198.000000,24.238908,8.000000,81.0,61.700000,5.0,38.0,24.0,0.000694,1.370443,35.600000,8.000000,88.000000,23.0,158.0,20.041341,9.000000,42.0,11.000000,60.0,3.2,34.8,185.000000,25.868737,34.80000,4.0,143.192832,165.1,1.038591,7.368395,5.000000,25.772063,185.000000,84.0,96.0,6.0,3.500000,116.0,1.430113,25.900000,11.600000,158.0,16.0,34.000000,8.180786,48.0,137.791179,100.0,106.0,138.939562,0.0,99.0,12.000000,84.0,97.000000,48.0,114.0,84.0,42.0,25.900000,74.0,8.900000,0
4,-0.040290,-0.017745,0.091518,96.0,23.756070,0.087740,31.474937,60.0,120.0,90.0,0.0,239.412553,16.0,38.0,36.7,15.0,238.901566,24.627807,9.822814,19.0,92.294235,4.0,37.2,29.0,0.073611,1.115602,36.990701,9.912383,110.312610,18.0,147.0,8.468848,9.798405,57.0,17.299467,103.0,0.5,36.7,146.845870,7.801982,36.99038,16.0,137.792999,188.0,1.099951,8.512058,3.768750,8.384335,95.679975,120.0,76.0,42.0,3.882038,89.0,1.096540,37.156718,12.418329,147.0,2.0,36.641

In [0]:
#save model
save_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/'
    
pkl_filename = str(save_path + "imp_78featH2O_mapping.pkl")  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(imp, file)


# test data


In [18]:
test_supplied = pd.read_csv(unlabeled_data_path)
test_supplied = pd.DataFrame(data = test_supplied)

print('Original data shape:\n', test_supplied.shape, '\n')
print('Group value counts:\n', test_supplied['hospital_death'].value_counts(), '\n')

test39_supplied = pd.read_csv(unlabeled39_data_path)
test39_supplied = pd.DataFrame(data = test39_supplied)

print('Original data shape:\n', test39_supplied.shape, '\n')
print('Group value counts:\n', test39_supplied['hospital_death'].value_counts(), '\n')

#create feature set 
test_supplied_imp1forimp2 = pd.concat([test39_supplied.drop(['Unnamed: 0', 'hospital_death', 'encounter_id'], axis=1), test_supplied[feat_H2O_second39]], axis=1)
print(test_supplied_imp1forimp2.shape)
test_supplied_imp1forimp2.head()

Original data shape:
 (39308, 154) 

Group value counts:
 Series([], Name: hospital_death, dtype: int64) 

Original data shape:
 (39308, 42) 

Group value counts:
 Series([], Name: hospital_death, dtype: int64) 

(39308, 78)


,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,apache_3j_diagnosis,d1_spo2_min,d1_hco3_min,apache_2_diagnosis,bmi,d1_heartrate_min,d1_sysbp_min,d1_mbp_min,ventilated_apache,d1_platelets_min,d1_resprate_min,param_10_diff_count,d1_temp_min,param_25_diff_count,d1_platelets_max,d1_hco3_max,wbc_apache,age,weight,d1_spo2_diff,d1_temp_max,heart_rate_diff,pre_icu_los_days,creatinine_apache,h1_temp_max,d1_wbc_min,d1_glucose_min,d1_resprate_max,d1_sysbp_max,d1_bun_min,d1_wbc_max,d1_diasbp_min,h1_resprate_min,map_apache,temp_diff,temp_apache,glucose_apache,d1_bun_max,h1_temp_min,resprate_apache,d1_sodium_max,height,d1_creatinine_min,d1_calcium_min,d1_potassium_max,bun_apache,d1_glucose_max,d1_sysbp_noninvasive_min,h1_heartrate_min,d1_diasbp_diff,d1_potassium_min,d1_heartrate_max,d1_creatinine_max,hematocrit_apache,d1_hemaglobin_max,d1_sysbp_noninvasive_max,resprate_diff,d1_hematocrit_max,d1_calcium_max,na_count,d1_sodium_min,h1_heartrate_max,h1_sysbp_min,sodium_apache,map_diff,h1_spo2_min,h1_resprate_max,d1_mbp_max,glucose_diff,d1_diasbp_max,heart_rate_apache,d1_mbp_noninvasive_min,d1_diasbp_noninvasive_min,d1_hematocrit_min,d1_sysbp_diff,d1_hemaglobin_min
0,0.13,0.08,0.032178,96.0,24.000000,0.072866,21.102439,78.0,79.0,60.0,0.0,223.0,8.0,34.0,36.6,15.0,223.0,24.000000,4.700000,56.000000,68.6,4.0,37.1,22.0,0.104167,0.670000,36.600000,4.7,167.0,20.0,121.0,19.0,4.7,51.0,8.000000,57.0,0.5,36.6,276.000000,19.0,36.6,5.0,135.0,180.3,0.67,9.3,4.8,19.0,276.0,79.0,84.0,23.0,4.8,100.0,0.67,42.0,13.8,121.0,12.0,42.0,9.3,71,134.0,88.0,79.0,134.0,26.0,98.0,18.0,86.0,109.0,74.0,105.0,60.0,51.0,42.0,42.0,13.8
1,0.09,0.03,0.172973,90.0,31.000000,0.136646,19.500595,60.0,83.0,53.0,0.0,93.0,16.0,33.0,36.6,15.0,93.0,31.000000,2.483438,80.922925,41.0,9.0,37.7,13.0,0.230556,0.455391,36.600000,3.0,100.0,33.0,148.0,12.0,3.0,31.0,16.000000,53.0,1.1,37.0,95.849031,12.0,36.6,37.0,140.0,145.0,0.60,8.0,4.4,NaN,100.0,83.0,60.0,34.0,4.4,73.0,0.60,NaN,9.9,148.0,17.0,30.9,8.0,77,140.0,60.0,96.0,NaN,55.0,92.0,16.0,108.0,0.0,65.0,60.0,53.0,31.0,30.9,65.0,9.9
2,0.03,0.02,0.043956,95.0,19.000000,0.035363,32.518596,86.0,102.0,71.0,0.0,181.0,10.0,31.0,35.5,15.0,184.0,22.000000,14.800000,66.000000,94.2,5.0,37.5,31.0,0.140972,1.840000,36.309482,10.0,81.0,28.0,172.0,19.0,14.8,57.0,14.800868,115.0,2.0,35.5,81.000000,22.0,NaN,31.0,135.0,170.2,1.63,8.4,4.7,22.0,117.0,NaN,NaN,28.0,4.5,117.0,1.84,39.2,12.7,NaN,18.0,40.9,8.7,94,130.0,NaN,NaN,130.0,39.0,NaN,NaN,110.0,36.0,85.0,124.0,NaN,NaN,39.2,70.0,12.2
3,0.19,0.17,0.178824,92.0,20.000000,0.095224,20.798520,60.0,69.0,48.0,1.0,209.0,15.0,12.0,37.5,0.0,248.0,22.000000,9.700000,64.000000,69.5,8.0,38.4,51.0,0.002083,0.750000,38.000000,7.1,122.0,35.0,115.0,24.0,9.7,42.0,23.000000,48.0,0.9,37.7,122.000000,24.0,38.0,30.0,138.0,182.8,0.72,7.4,4.1,24.0,148.0,69.0,102.0,21.0,4.0,111.0,0.75,32.0,11.0,115.0,20.0,32.0,7.9,23,133.0,111.0,92.0,133.0,28.0,93.0,26.0,76.0,26.0,63.0,111.0,48.0,42.0,27.0,46.0,9.1
4,0.50,0.42,0.256881,95.0,21.320281,0.136646,24.766867,62.0,66.0,37.0,1.0,218.0,0.0,23.0,36.0,15.0,223.0,22.770291,14.500000,56.000000,66.4,5.0,37.9,32.0,0.004861,0.520000,36.000000,14.1,141.0,21.0,222.0,9.0,14.5,24.0,16.000000,41.0,1.9,36.1,192.000000,11.0,36.0,5.0,150.0,NaN,0.52,7.7,4.2,11.0,192.0,66.0,76.0,76.0,3.5,94.0,0.67,32.8,12.3,222.0,21.0,36.5,8.2,65,134.0,82.0,194.0,134.0,100.0,99.0,16.0,137.0,51.0,100.0,94.0,37.0,24.0,32.8,156.0,11.1


In [19]:
data = test_supplied_imp1forimp2

test_supplied_imp = imp.transform(data)

#create final data set
final_data_test = pd.DataFrame(data=test_supplied_imp, columns=data.columns.values)
#put back correct hospital death
final_data_test['hospital_death'] = test_supplied['hospital_death']
final_data_test['encounter_id'] = test_supplied['encounter_id']

final_data_test.head()

[IterativeImputer] Completing matrix with shape (39308, 78)
[IterativeImputer] Ending imputation round 1/99, elapsed time 0.24
[IterativeImputer] Ending imputation round 2/99, elapsed time 0.47
[IterativeImputer] Ending imputation round 3/99, elapsed time 0.69
[IterativeImputer] Ending imputation round 4/99, elapsed time 0.93
[IterativeImputer] Ending imputation round 5/99, elapsed time 1.19
[IterativeImputer] Ending imputation round 6/99, elapsed time 1.42
[IterativeImputer] Ending imputation round 7/99, elapsed time 1.66
[IterativeImputer] Ending imputation round 8/99, elapsed time 1.89
[IterativeImputer] Ending imputation round 9/99, elapsed time 2.11
[IterativeImputer] Ending imputation round 10/99, elapsed time 2.35
[IterativeImputer] Ending imputation round 11/99, elapsed time 2.58
[IterativeImputer] Ending imputation round 12/99, elapsed time 2.80
[IterativeImputer] Ending imputation round 13/99, elapsed time 3.03
[IterativeImputer] Ending imputation round 14/99, elapsed time 3.

,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,apache_3j_diagnosis,d1_spo2_min,d1_hco3_min,apache_2_diagnosis,bmi,d1_heartrate_min,d1_sysbp_min,d1_mbp_min,ventilated_apache,d1_platelets_min,d1_resprate_min,param_10_diff_count,d1_temp_min,param_25_diff_count,d1_platelets_max,d1_hco3_max,wbc_apache,age,weight,d1_spo2_diff,d1_temp_max,heart_rate_diff,pre_icu_los_days,creatinine_apache,h1_temp_max,d1_wbc_min,d1_glucose_min,d1_resprate_max,d1_sysbp_max,d1_bun_min,d1_wbc_max,d1_diasbp_min,h1_resprate_min,map_apache,temp_diff,temp_apache,glucose_apache,d1_bun_max,h1_temp_min,resprate_apache,d1_sodium_max,height,d1_creatinine_min,d1_calcium_min,d1_potassium_max,bun_apache,d1_glucose_max,d1_sysbp_noninvasive_min,h1_heartrate_min,d1_diasbp_diff,d1_potassium_min,d1_heartrate_max,d1_creatinine_max,hematocrit_apache,d1_hemaglobin_max,d1_sysbp_noninvasive_max,resprate_diff,d1_hematocrit_max,d1_calcium_max,na_count,d1_sodium_min,h1_heartrate_max,h1_sysbp_min,sodium_apache,map_diff,h1_spo2_min,h1_resprate_max,d1_mbp_max,glucose_diff,d1_diasbp_max,heart_rate_apache,d1_mbp_noninvasive_min,d1_diasbp_noninvasive_min,d1_hematocrit_min,d1_sysbp_diff,d1_hemaglobin_min,hospital_death,encounter_id
0,0.13,0.08,0.032178,96.0,24.000000,0.072866,21.102439,78.0,79.0,60.0,0.0,223.0,8.0,34.0,36.6,15.0,223.0,24.000000,4.700000,56.000000,68.6,4.0,37.1,22.0,0.104167,0.670000,36.600000,4.7,167.0,20.0,121.0,19.0,4.7,51.0,8.000000,57.0,0.5,36.6,276.000000,19.0,36.600000,5.0,135.0,180.300000,0.67,9.3,4.8,19.000000,276.0,79.000000,84.000000,23.0,4.8,100.0,0.67,42.000000,13.8,121.000000,12.0,42.0,9.3,71.0,134.0,88.000000,79.000000,134.000000,26.0,98.000000,18.000000,86.0,109.0,74.0,105.0,60.000000,51.000000,42.0,42.0,13.8,NaN,2
1,0.09,0.03,0.172973,90.0,31.000000,0.136646,19.500595,60.0,83.0,53.0,0.0,93.0,16.0,33.0,36.6,15.0,93.0,31.000000,2.483438,80.922925,41.0,9.0,37.7,13.0,0.230556,0.455391,36.600000,3.0,100.0,33.0,148.0,12.0,3.0,31.0,16.000000,53.0,1.1,37.0,95.849031,12.0,36.600000,37.0,140.0,145.000000,0.60,8.0,4.4,10.884289,100.0,83.000000,60.000000,34.0,4.4,73.0,0.60,30.893688,9.9,148.000000,17.0,30.9,8.0,77.0,140.0,60.000000,96.000000,140.128845,55.0,92.000000,16.000000,108.0,0.0,65.0,60.0,53.000000,31.000000,30.9,65.0,9.9,NaN,5
2,0.03,0.02,0.043956,95.0,19.000000,0.035363,32.518596,86.0,102.0,71.0,0.0,181.0,10.0,31.0,35.5,15.0,184.0,22.000000,14.800000,66.000000,94.2,5.0,37.5,31.0,0.140972,1.840000,36.309482,10.0,81.0,28.0,172.0,19.0,14.8,57.0,14.800868,115.0,2.0,35.5,81.000000,22.0,36.131982,31.0,135.0,170.200000,1.63,8.4,4.7,22.000000,117.0,102.039941,93.570707,28.0,4.5,117.0,1.84,39.200000,12.7,171.755685,18.0,40.9,8.7,94.0,130.0,101.094001,130.406228,130.000000,39.0,96.818084,21.237071,110.0,36.0,85.0,124.0,71.023245,57.038519,39.2,70.0,12.2,NaN,7
3,0.19,0.17,0.178824,92.0,20.000000,0.095224,20.798520,60.0,69.0,48.0,1.0,209.0,15.0,12.0,37.5,0.0,248.0,22.000000,9.700000,64.000000,69.5,8.0,38.4,51.0,0.002083,0.750000,38.000000,7.1,122.0,35.0,115.0,24.0,9.7,42.0,23.000000,48.0,0.9,37.7,122.000000,24.0,38.000000,30.0,138.0,182.800000,0.72,7.4,4.1,24.000000,148.0,69.000000,102.000000,21.0,4.0,111.0,0.75,32.000000,11.0,115.000000,20.0,32.0,7.9,23.0,133.0,111.000000,92.000000,133.000000,28.0,93.000000,26.000000,76.0,26.0,63.0,111.0,48.000000,42.000000,27.0,46.0,9.1,NaN,8
4,0.50,0.42,0.256881,95.0,21.320281,0.136646,24.766867,62.0,66.0,37.0,1.0,218.0,0.0,23.0,36.0,15.0,223.0,22.770291,14.500000,56.000000,66.4,5.0,37.9,32.0,0.004861,0.520000,36.000000,14.1,141.0,21.0,222.0,9.0,14.5,24.0,16.000000,41.0,1.9,36.1,192.000000,11.0,36.000000,5.0,150.0,164.513555,0.52,7.7,4.2,11.000000,192.0,66.000000,76.000000,76.0,3.5,94.0,0.67,32.800000,12.3,222.000000,21.0,36.5,8.2,65.0,134.0,82.000000,194.000000,134.000000,100.0,99.000000,16.000000,137.0,51.0,100.0,94.0,37.000000,24.000000,32.8,156.0,11.1,NaN,10


In [0]:
final_data_train.to_csv(save_path + 'train_data_78featH2O_impute.csv')
final_data_test.to_csv(save_path + 'test_data_78featH2O_impute.csv')